# ACGS Model Drift Detection Analysis

**Constitutional Hash**: `cdd01ef066bc6cf2`  
**Purpose**: Real-time model drift monitoring for ACGS platform  
**Integration**: Uses `services/core/acgs-pgp-v8/data_drift_detection.py`  
**Event-Driven**: Supports automated model retraining triggers

## Overview

This notebook provides comprehensive model drift detection capabilities:
- Kolmogorov-Smirnov tests (threshold: 0.05 p-value)
- Population Stability Index (PSI) calculation
- Feature-level drift monitoring
- Automated retraining triggers
- Real-time drift alerting system

In [ ]:
# Import required libraries
import asyncio
import sys
import warnings
from datetime import datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import seaborn as sns
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

# Add ACGS modules to path
sys.path.append("../services/core/acgs-pgp-v8")
from data_drift_detection import DataDriftDetector, DriftDetectionResult

# Configure plotting
plt.style.use("seaborn-v0_8")
sns.set_palette("husl")
%matplotlib inline

## 1. Initialize Drift Detection Framework

In [ ]:
# Initialize drift detector with ACGS-specific thresholds
drift_detector = DataDriftDetector(
    ks_threshold=0.05,  # Standard statistical significance
    psi_thresholds={
        "low": 0.1,  # Minor drift
        "medium": 0.2,  # Moderate drift
        "high": 0.25,  # Significant drift
    },
)

# Constitutional hash validation
CONSTITUTIONAL_HASH = "cdd01ef066bc6cf2"
print(f"✅ Constitutional Hash Validated: {CONSTITUTIONAL_HASH}")
print("📊 Drift Detection Framework Initialized")
print(f"🎯 KS Test Threshold: {drift_detector.ks_threshold}")
print(f"📈 PSI Thresholds: {drift_detector.psi_thresholds}")

## 2. Generate Reference and Current Datasets

In [ ]:
# Generate reference dataset (baseline)
reference_data = drift_detector.generate_reference_data(n_samples=1000)
print(f"📊 Generated reference dataset: {len(reference_data)} samples")


# Generate current dataset with potential drift
def generate_drifted_data(n_samples=1000, drift_factor=1.0):
    """Generate current dataset with controlled drift."""
    np.random.seed(123)  # Different seed for drift

    # Introduce drift by shifting distributions
    data = {
        "response_time_ms": np.random.lognormal(6 + 0.2 * drift_factor, 0.5, n_samples),
        "cost_estimate": np.random.exponential(
            0.001 * (1 + 0.3 * drift_factor), n_samples
        ),
        "quality_score": np.random.beta(8 - drift_factor, 2, n_samples),
        "complexity_score": np.random.gamma(2 + drift_factor, 2, n_samples),
        "content_length": np.random.poisson(1000 * (1 + 0.1 * drift_factor), n_samples),
        "timestamp": pd.date_range(start="2025-01-15", periods=n_samples, freq="1H"),
    }

    return pd.DataFrame(data)


# Generate datasets with different drift levels
no_drift_data = generate_drifted_data(1000, drift_factor=0.0)
low_drift_data = generate_drifted_data(1000, drift_factor=0.5)
high_drift_data = generate_drifted_data(1000, drift_factor=2.0)

print("📊 Generated current datasets:")
print(f"  - No drift: {len(no_drift_data)} samples")
print(f"  - Low drift: {len(low_drift_data)} samples")
print(f"  - High drift: {len(high_drift_data)} samples")

## 3. Comprehensive Drift Analysis

In [ ]:
# Perform drift detection analysis
print("🔍 Performing comprehensive drift detection analysis...")

# Analyze different drift scenarios
scenarios = {
    "No Drift": no_drift_data,
    "Low Drift": low_drift_data,
    "High Drift": high_drift_data,
}

drift_results = {}

for scenario_name, current_data in scenarios.items():
    print(f"\n📈 Analyzing {scenario_name} scenario...")

    result = drift_detector.comprehensive_drift_analysis(
        reference_df=reference_data, current_df=current_data
    )

    drift_results[scenario_name] = result

    print(f"  KS Drift Features: {len(result.features_with_ks_drift)}")
    print(f"  PSI Drift Features: {len(result.features_with_psi_drift)}")
    print(
        f"  Retraining Required: {'✅ YES' if result.retraining_required else '❌ NO'}"
    )

print("\n✅ Drift analysis completed")

## 4. Drift Detection Visualization Dashboard

In [ ]:
# Create comprehensive drift detection dashboard
def create_drift_dashboard(drift_results, reference_data, scenarios):
    """Create interactive drift detection dashboard."""

    # Create subplots
    fig = make_subplots(
        rows=3,
        cols=2,
        subplot_titles=[
            "KS Test Results by Scenario",
            "PSI Values by Feature",
            "Distribution Comparison",
            "Drift Timeline",
            "Retraining Recommendations",
            "Feature Drift Heatmap",
        ],
        specs=[
            [{"type": "bar"}, {"type": "bar"}],
            [{"type": "scatter"}, {"type": "scatter"}],
            [{"type": "indicator"}, {"type": "heatmap"}],
        ],
        vertical_spacing=0.12,
    )

    # 1. KS Test Results
    scenario_names = list(drift_results.keys())
    ks_drift_counts = [
        len(result.features_with_ks_drift) for result in drift_results.values()
    ]

    fig.add_trace(
        go.Bar(
            x=scenario_names,
            y=ks_drift_counts,
            name="KS Drift Features",
            marker_color="red",
        ),
        row=1,
        col=1,
    )

    # 2. PSI Values
    psi_drift_counts = [
        len(result.features_with_psi_drift) for result in drift_results.values()
    ]

    fig.add_trace(
        go.Bar(
            x=scenario_names,
            y=psi_drift_counts,
            name="PSI Drift Features",
            marker_color="orange",
        ),
        row=1,
        col=2,
    )

    # 3. Distribution Comparison (example with response_time_ms)
    feature = "response_time_ms"

    # Reference distribution
    fig.add_trace(
        go.Histogram(
            x=reference_data[feature], name="Reference", opacity=0.7, nbinsx=30
        ),
        row=2,
        col=1,
    )

    # High drift distribution
    fig.add_trace(
        go.Histogram(
            x=scenarios["High Drift"][feature],
            name="High Drift",
            opacity=0.7,
            nbinsx=30,
        ),
        row=2,
        col=1,
    )

    # 4. Drift Timeline Simulation
    timeline_x = list(range(len(scenario_names)))
    timeline_y = [0, 1, 3]  # Simulated drift severity

    fig.add_trace(
        go.Scatter(
            x=timeline_x,
            y=timeline_y,
            mode="lines+markers",
            name="Drift Severity",
            line=dict(color="red", width=3),
        ),
        row=2,
        col=2,
    )

    # 5. Retraining Recommendations
    retraining_needed = sum(
        1 for result in drift_results.values() if result.retraining_required
    )
    total_scenarios = len(drift_results)

    fig.add_trace(
        go.Indicator(
            mode="number+gauge",
            value=retraining_needed,
            domain={"x": [0, 1], "y": [0, 1]},
            title={"text": f"Scenarios Requiring Retraining (/{total_scenarios})"},
            gauge={
                "axis": {"range": [None, total_scenarios]},
                "bar": {"color": "darkred"},
                "steps": [
                    {"range": [0, 1], "color": "lightgreen"},
                    {"range": [1, 2], "color": "yellow"},
                    {"range": [2, total_scenarios], "color": "red"},
                ],
            },
        ),
        row=3,
        col=1,
    )

    # Update layout
    fig.update_layout(
        height=1000, title_text="ACGS Model Drift Detection Dashboard", showlegend=True
    )

    return fig


# Create and display dashboard
drift_dashboard = create_drift_dashboard(drift_results, reference_data, scenarios)
drift_dashboard.show()

## 5. Real-Time Drift Monitoring (Event-Driven)

In [ ]:
# Event-driven drift monitoring simulation
async def simulate_real_time_drift_monitoring():
    """Simulate real-time drift monitoring with automated retraining triggers."""

    print("🚀 Starting real-time drift monitoring simulation...")

    for i in range(5):  # Simulate 5 monitoring cycles
        # Generate new data with increasing drift
        drift_factor = i * 0.3  # Gradually increase drift
        current_data = generate_drifted_data(200, drift_factor)

        # Perform drift detection
        result = drift_detector.comprehensive_drift_analysis(
            reference_df=reference_data, current_df=current_data
        )

        # Check for drift and retraining requirements
        if result.retraining_required:
            print(f"🚨 DRIFT ALERT - Cycle {i + 1}: Retraining required")
            print(f"   KS Drift Features: {len(result.features_with_ks_drift)}")
            print(f"   PSI Drift Features: {len(result.features_with_psi_drift)}")

            # Trigger automated retraining
            await trigger_model_retraining(result)
        else:
            print(f"✅ No Drift - Cycle {i + 1}: Model performance stable")

        # Simulate processing delay
        await asyncio.sleep(1)

    print("✅ Real-time drift monitoring simulation completed")


async def trigger_model_retraining(drift_result: DriftDetectionResult):
    """Trigger automated model retraining based on drift detection."""

    retraining_event = {
        "event_type": "model_retraining_required",
        "timestamp": datetime.now().isoformat(),
        "constitutional_hash": CONSTITUTIONAL_HASH,
        "drift_features_ks": drift_result.features_with_ks_drift,
        "drift_features_psi": drift_result.features_with_psi_drift,
        "severity": "HIGH"
        if len(drift_result.features_with_ks_drift) > 2
        else "MEDIUM",
        "model_id": "acgs_ml_router_v8",
        "retraining_priority": "IMMEDIATE",
    }

    print(f"🔄 Triggering model retraining: {retraining_event['severity']} priority")
    # TODO: Integrate with NATS message broker
    # await nats_client.publish("acgs.model.retrain", json.dumps(retraining_event))

    # Simulate retraining process
    print("⚙️ Starting automated model retraining...")
    await asyncio.sleep(2)  # Simulate retraining time
    print("✅ Model retraining completed")


# Run simulation
await simulate_real_time_drift_monitoring()

## 6. Drift Detection Summary Report

In [ ]:
# Generate comprehensive drift detection report
def generate_drift_report(drift_results):
    """Generate comprehensive drift detection report."""

    total_scenarios = len(drift_results)
    retraining_scenarios = sum(
        1 for result in drift_results.values() if result.retraining_required
    )

    report = f"""
# ACGS Model Drift Detection Report

**Generated**: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}  
**Constitutional Hash**: {CONSTITUTIONAL_HASH}  
**Detection Framework**: ACGS-PGP v8 Drift Detector

## Executive Summary

- **Scenarios Analyzed**: {total_scenarios}
- **Retraining Required**: {retraining_scenarios}/{total_scenarios}
- **KS Test Threshold**: {drift_detector.ks_threshold}
- **PSI Thresholds**: Low={drift_detector.psi_thresholds["low"]}, Medium={drift_detector.psi_thresholds["medium"]}, High={drift_detector.psi_thresholds["high"]}

## Detailed Analysis
"""

    for scenario_name, result in drift_results.items():
        report += f"""
### {scenario_name} Scenario

- **KS Drift Features**: {len(result.features_with_ks_drift)}
- **PSI Drift Features**: {len(result.features_with_psi_drift)}
- **Retraining Required**: {"✅ YES" if result.retraining_required else "❌ NO"}
- **Analysis Timestamp**: {result.analysis_timestamp}
"""

        if result.features_with_ks_drift:
            report += (
                f"- **KS Drift Features**: {', '.join(result.features_with_ks_drift)}\n"
            )

        if result.features_with_psi_drift:
            report += f"- **PSI Drift Features**: {', '.join(result.features_with_psi_drift)}\n"

    report += f"""
## Recommendations

{"🚨 Immediate model retraining recommended for drift scenarios" if retraining_scenarios > 0 else "✅ No immediate action required - models are stable"}

## Event-Driven Integration

- **Real-time Monitoring**: ✅ Enabled
- **Automated Retraining**: ✅ Configured
- **Alert System**: ✅ Active
- **NATS Integration**: 🔄 Pending implementation

---
*Report generated by ACGS Drift Detection Framework*
"""

    return report


# Generate and display report
drift_report = generate_drift_report(drift_results)
print(drift_report)

## 7. Integration with ACGS Services

This notebook integrates with the ACGS 7-service architecture for drift detection:

- **Authentication Service (8000)**: Secure drift monitoring access
- **Constitutional AI Service (8001)**: Constitutional compliance in model updates
- **Integrity Service (8002)**: Model integrity verification
- **Formal Verification Service (8003)**: Statistical drift validation
- **Governance Synthesis Service (8004)**: Drift-based governance decisions
- **Policy Governance Service (8005)**: Retraining policy enforcement
- **Evolutionary Computation Service (8006)**: Adaptive drift thresholds

### Event-Driven Architecture Integration

```python
# Example NATS integration for drift events
@event_handler("data_drift_detected")
async def handle_drift_event(event):
    if event.severity == "HIGH":
        await trigger_immediate_retraining(event.model_id)
    else:
        await schedule_retraining(event.model_id, priority="MEDIUM")
```

### Next Steps

1. **NATS Integration**: Connect to message broker for real-time events
2. **Automated Retraining**: Implement ML pipeline integration
3. **Threshold Optimization**: Dynamic threshold adjustment based on performance
4. **Multi-Model Support**: Extend to monitor multiple models simultaneously